In [1]:
import geodist
import web_scrapping
import tratamento_base_climatica
from pandas import DataFrame, merge
import os
import numpy as np
from diblogeo import DistanceUnits

In [2]:
def recuperar_tuplas_somente_de_aeroportos(soup, caminho_para_trs_da_table):
    tuplas = web_scrapping.recuperar_data_frame_de_tuplas(soup, caminho_para_trs_da_table)
    tuplas = tuplas.loc[tuplas['ICAO:'] != '-']
    return tuplas

In [3]:
#Recupera dataframes com as informações dos aeroportos
dataframe_aeroportos = recuperar_tuplas_somente_de_aeroportos('resource/html_geolocation_aeroportos.html', 'table.tabl tr')


In [4]:
#Recupera dataframe com informações climáticos por WMO
dataframe_estacao_climatica = tratamento_base_climatica.recuperar_data_frame_de_tuplas('resource/climahistorico/')

In [5]:
#Realiza o Join entre as informações de aeroportos com informações climáticas
df_resultante = merge(dataframe_aeroportos, dataframe_estacao_climatica, left_on = 'WMO:', right_on = 'Estacao', how = 'left')

In [6]:
#Recupera aeroportos que não foram encontrados na lista de wmos
aeroportos_sem_estacao_climatica_por_wmo = df_resultante.loc[df_resultante["Estacao"].isnull()]

In [7]:
#Recupera um mapa de wmo's com sua latitude e longitude
df_wmos_lat_long = tratamento_base_climatica.recuperar_wmos_lat_long('resource/climahistorico/')

In [8]:
#Realiza o calculo de distancia de cada aeroporto sem wmo associado e recupera um mapa com a informação
#do wmo mais próximo encontrado para cada aeroporto sem essa informação
mapa_wmo_proximo_por_aeroporto = dict()
for i,aeroporto in aeroportos_sem_estacao_climatica_por_wmo.iterrows():
    lat_long_aeroporto = (float(aeroporto["Latitude:"]), float(aeroporto["Longitude:"]))
    wmo_com_menor_distancia = None
    menor_distancia = 0
    for index, wmo in df_wmos_lat_long.iterrows():
        lat_long_wmo = (wmo['Latitude'], wmo['Longitude'])
        print ('Aeroporto: ', lat_long_aeroporto, ' WMO: ', lat_long_wmo)
        distancia = geodist.calcular_distancia_geoespacial(lat_long_aeroporto, lat_long_wmo)
        if menor_distancia == 0 or distancia.meters < menor_distancia.meters:
            menor_distancia = distancia
            wmo_com_menor_distancia = wmo['Estacao']
    icao = aeroporto['ICAO:']
    print('O WMO {} é o mais próximo ao aeroporto de ICAO {}, com distância geoespacial de {}'.format(wmo_com_menor_distancia, icao,menor_distancia))
    mapa_wmo_proximo_por_aeroporto[icao] = wmo_com_menor_distancia

Aeroporto:  (3.84, -51.82)  WMO:  (-3.21, -52.21)
Aeroporto:  (3.84, -51.82)  WMO:  (-10.95, -37.04)
Aeroporto:  (3.84, -51.82)  WMO:  (-1.43, -48.43)
Aeroporto:  (3.84, -51.82)  WMO:  (-19.93, -43.93)
Aeroporto:  (3.84, -51.82)  WMO:  (2.82, -60.66)
Aeroporto:  (3.84, -51.82)  WMO:  (-13.26, -43.41)
Aeroporto:  (3.84, -51.82)  WMO:  (-15.78, -47.92)
Aeroporto:  (3.84, -51.82)  WMO:  (-7.22, -35.88)
Aeroporto:  (3.84, -51.82)  WMO:  (1.83, -61.12)
Aeroporto:  (3.84, -51.82)  WMO:  (-7.6, -72.66)
Aeroporto:  (3.84, -51.82)  WMO:  (-15.61, -56.1)
Aeroporto:  (3.84, -51.82)  WMO:  (-25.43, -49.26)
Aeroporto:  (3.84, -51.82)  WMO:  (-25.43, -49.26)
Aeroporto:  (3.84, -51.82)  WMO:  (-27.58, -48.56)
Aeroporto:  (3.84, -51.82)  WMO:  (-3.81, -38.53)
Aeroporto:  (3.84, -51.82)  WMO:  (-16.66, -49.25)
Aeroporto:  (3.84, -51.82)  WMO:  (-5.53, -47.48)
Aeroporto:  (3.84, -51.82)  WMO:  (-7.1, -34.86)
Aeroporto:  (3.84, -51.82)  WMO:  (-23.31, -51.13)
Aeroporto:  (3.84, -51.82)  WMO:  (-0.05, -51

Aeroporto:  (-0.89, -52.6)  WMO:  (-16.66, -49.25)
Aeroporto:  (-0.89, -52.6)  WMO:  (-5.53, -47.48)
Aeroporto:  (-0.89, -52.6)  WMO:  (-7.1, -34.86)
Aeroporto:  (-0.89, -52.6)  WMO:  (-23.31, -51.13)
Aeroporto:  (-0.89, -52.6)  WMO:  (-0.05, -51.11)
Aeroporto:  (-0.89, -52.6)  WMO:  (-0.05, -51.11)
Aeroporto:  (-0.89, -52.6)  WMO:  (-9.66, -35.7)
Aeroporto:  (-0.89, -52.6)  WMO:  (-3.1, -60.01)
Aeroporto:  (-0.89, -52.6)  WMO:  (-5.36, -49.13)
Aeroporto:  (-0.89, -52.6)  WMO:  (-16.68, -43.84)
Aeroporto:  (-0.89, -52.6)  WMO:  (-5.91, -35.2)
Aeroporto:  (-0.89, -52.6)  WMO:  (-10.19, -48.3)
Aeroporto:  (-0.89, -52.6)  WMO:  (-3.08, -41.76)
Aeroporto:  (-0.89, -52.6)  WMO:  (-9.36, -38.21)
Aeroporto:  (-0.89, -52.6)  WMO:  (-31.78, -52.41)
Aeroporto:  (-0.89, -52.6)  WMO:  (-9.38, -40.48)
Aeroporto:  (-0.89, -52.6)  WMO:  (-22.55, -55.71)
Aeroporto:  (-0.89, -52.6)  WMO:  (-30.05, -51.16)
Aeroporto:  (-0.89, -52.6)  WMO:  (-8.05, -34.95)
Aeroporto:  (-0.89, -52.6)  WMO:  (-9.96, -67.8)

O WMO 82353 é o mais próximo ao aeroporto de ICAO SBSN, com distância geoespacial de 298.9685124043127
Aeroporto:  (-2.32, -44.42)  WMO:  (-3.21, -52.21)
Aeroporto:  (-2.32, -44.42)  WMO:  (-10.95, -37.04)
Aeroporto:  (-2.32, -44.42)  WMO:  (-1.43, -48.43)
Aeroporto:  (-2.32, -44.42)  WMO:  (-19.93, -43.93)
Aeroporto:  (-2.32, -44.42)  WMO:  (2.82, -60.66)
Aeroporto:  (-2.32, -44.42)  WMO:  (-13.26, -43.41)
Aeroporto:  (-2.32, -44.42)  WMO:  (-15.78, -47.92)
Aeroporto:  (-2.32, -44.42)  WMO:  (-7.22, -35.88)
Aeroporto:  (-2.32, -44.42)  WMO:  (1.83, -61.12)
Aeroporto:  (-2.32, -44.42)  WMO:  (-7.6, -72.66)
Aeroporto:  (-2.32, -44.42)  WMO:  (-15.61, -56.1)
Aeroporto:  (-2.32, -44.42)  WMO:  (-25.43, -49.26)
Aeroporto:  (-2.32, -44.42)  WMO:  (-25.43, -49.26)
Aeroporto:  (-2.32, -44.42)  WMO:  (-27.58, -48.56)
Aeroporto:  (-2.32, -44.42)  WMO:  (-3.81, -38.53)
Aeroporto:  (-2.32, -44.42)  WMO:  (-16.66, -49.25)
Aeroporto:  (-2.32, -44.42)  WMO:  (-5.53, -47.48)
Aeroporto:  (-2.32, -44.4

Aeroporto:  (-3.76, -49.67)  WMO:  (-9.36, -38.21)
Aeroporto:  (-3.76, -49.67)  WMO:  (-31.78, -52.41)
Aeroporto:  (-3.76, -49.67)  WMO:  (-9.38, -40.48)
Aeroporto:  (-3.76, -49.67)  WMO:  (-22.55, -55.71)
Aeroporto:  (-3.76, -49.67)  WMO:  (-30.05, -51.16)
Aeroporto:  (-3.76, -49.67)  WMO:  (-8.05, -34.95)
Aeroporto:  (-3.76, -49.67)  WMO:  (-9.96, -67.8)
Aeroporto:  (-3.76, -49.67)  WMO:  (-22.89, -43.18)
Aeroporto:  (-3.76, -49.67)  WMO:  (-13.01, -38.53)
Aeroporto:  (-3.76, -49.67)  WMO:  (-2.53, -44.21)
Aeroporto:  (-3.76, -49.67)  WMO:  (-23.5, -46.61)
Aeroporto:  (-3.76, -49.67)  WMO:  (-3.83, -64.7)
Aeroporto:  (-3.76, -49.67)  WMO:  (-5.08, -42.81)
Aeroporto:  (-3.76, -49.67)  WMO:  (-19.73, -47.95)
Aeroporto:  (-3.76, -49.67)  WMO:  (-29.75, -57.08)
Aeroporto:  (-3.76, -49.67)  WMO:  (-20.31, -40.31)
O WMO 82562 é o mais próximo ao aeroporto de ICAO SBTU, com distância geoespacial de 187.91759387573126
Aeroporto:  (-3.77, -38.52)  WMO:  (-3.21, -52.21)
Aeroporto:  (-3.77, -38

Aeroporto:  (-4.24, -56.0)  WMO:  (-25.43, -49.26)
Aeroporto:  (-4.24, -56.0)  WMO:  (-27.58, -48.56)
Aeroporto:  (-4.24, -56.0)  WMO:  (-3.81, -38.53)
Aeroporto:  (-4.24, -56.0)  WMO:  (-16.66, -49.25)
Aeroporto:  (-4.24, -56.0)  WMO:  (-5.53, -47.48)
Aeroporto:  (-4.24, -56.0)  WMO:  (-7.1, -34.86)
Aeroporto:  (-4.24, -56.0)  WMO:  (-23.31, -51.13)
Aeroporto:  (-4.24, -56.0)  WMO:  (-0.05, -51.11)
Aeroporto:  (-4.24, -56.0)  WMO:  (-0.05, -51.11)
Aeroporto:  (-4.24, -56.0)  WMO:  (-9.66, -35.7)
Aeroporto:  (-4.24, -56.0)  WMO:  (-3.1, -60.01)
Aeroporto:  (-4.24, -56.0)  WMO:  (-5.36, -49.13)
Aeroporto:  (-4.24, -56.0)  WMO:  (-16.68, -43.84)
Aeroporto:  (-4.24, -56.0)  WMO:  (-5.91, -35.2)
Aeroporto:  (-4.24, -56.0)  WMO:  (-10.19, -48.3)
Aeroporto:  (-4.24, -56.0)  WMO:  (-3.08, -41.76)
Aeroporto:  (-4.24, -56.0)  WMO:  (-9.36, -38.21)
Aeroporto:  (-4.24, -56.0)  WMO:  (-31.78, -52.41)
Aeroporto:  (-4.24, -56.0)  WMO:  (-9.38, -40.48)
Aeroporto:  (-4.24, -56.0)  WMO:  (-22.55, -55.7

...

Aeroporto:  (-24.32, -50.65)  WMO:  (-22.89, -43.18)
Aeroporto:  (-24.32, -50.65)  WMO:  (-13.01, -38.53)
Aeroporto:  (-24.32, -50.65)  WMO:  (-2.53, -44.21)
Aeroporto:  (-24.32, -50.65)  WMO:  (-23.5, -46.61)
Aeroporto:  (-24.32, -50.65)  WMO:  (-3.83, -64.7)
Aeroporto:  (-24.32, -50.65)  WMO:  (-5.08, -42.81)
Aeroporto:  (-24.32, -50.65)  WMO:  (-19.73, -47.95)
Aeroporto:  (-24.32, -50.65)  WMO:  (-29.75, -57.08)
Aeroporto:  (-24.32, -50.65)  WMO:  (-20.31, -40.31)
O WMO 83766 é o mais próximo ao aeroporto de ICAO SBTL, com distância geoespacial de 122.53022188719535
Aeroporto:  (-24.68, -53.7)  WMO:  (-3.21, -52.21)
Aeroporto:  (-24.68, -53.7)  WMO:  (-10.95, -37.04)
Aeroporto:  (-24.68, -53.7)  WMO:  (-1.43, -48.43)
Aeroporto:  (-24.68, -53.7)  WMO:  (-19.93, -43.93)
Aeroporto:  (-24.68, -53.7)  WMO:  (2.82, -60.66)
Aeroporto:  (-24.68, -53.7)  WMO:  (-13.26, -43.41)
Aeroporto:  (-24.68, -53.7)  WMO:  (-15.78, -47.92)
Aeroporto:  (-24.68, -53.7)  WMO:  (-7.22, -35.88)
Aeroporto:  (

Aeroporto:  (-29.2, -51.18)  WMO:  (-0.05, -51.11)
Aeroporto:  (-29.2, -51.18)  WMO:  (-9.66, -35.7)
Aeroporto:  (-29.2, -51.18)  WMO:  (-3.1, -60.01)
Aeroporto:  (-29.2, -51.18)  WMO:  (-5.36, -49.13)
Aeroporto:  (-29.2, -51.18)  WMO:  (-16.68, -43.84)
Aeroporto:  (-29.2, -51.18)  WMO:  (-5.91, -35.2)
Aeroporto:  (-29.2, -51.18)  WMO:  (-10.19, -48.3)
Aeroporto:  (-29.2, -51.18)  WMO:  (-3.08, -41.76)
Aeroporto:  (-29.2, -51.18)  WMO:  (-9.36, -38.21)
Aeroporto:  (-29.2, -51.18)  WMO:  (-31.78, -52.41)
Aeroporto:  (-29.2, -51.18)  WMO:  (-9.38, -40.48)
Aeroporto:  (-29.2, -51.18)  WMO:  (-22.55, -55.71)
Aeroporto:  (-29.2, -51.18)  WMO:  (-30.05, -51.16)
Aeroporto:  (-29.2, -51.18)  WMO:  (-8.05, -34.95)
Aeroporto:  (-29.2, -51.18)  WMO:  (-9.96, -67.8)
Aeroporto:  (-29.2, -51.18)  WMO:  (-22.89, -43.18)
Aeroporto:  (-29.2, -51.18)  WMO:  (-13.01, -38.53)
Aeroporto:  (-29.2, -51.18)  WMO:  (-2.53, -44.21)
Aeroporto:  (-29.2, -51.18)  WMO:  (-23.5, -46.61)
Aeroporto:  (-29.2, -51.18)  

Aeroporto:  (-21.8, -48.13)  WMO:  (2.82, -60.66)
Aeroporto:  (-21.8, -48.13)  WMO:  (-13.26, -43.41)
Aeroporto:  (-21.8, -48.13)  WMO:  (-15.78, -47.92)
Aeroporto:  (-21.8, -48.13)  WMO:  (-7.22, -35.88)
Aeroporto:  (-21.8, -48.13)  WMO:  (1.83, -61.12)
Aeroporto:  (-21.8, -48.13)  WMO:  (-7.6, -72.66)
Aeroporto:  (-21.8, -48.13)  WMO:  (-15.61, -56.1)
Aeroporto:  (-21.8, -48.13)  WMO:  (-25.43, -49.26)
Aeroporto:  (-21.8, -48.13)  WMO:  (-25.43, -49.26)
Aeroporto:  (-21.8, -48.13)  WMO:  (-27.58, -48.56)
Aeroporto:  (-21.8, -48.13)  WMO:  (-3.81, -38.53)
Aeroporto:  (-21.8, -48.13)  WMO:  (-16.66, -49.25)
Aeroporto:  (-21.8, -48.13)  WMO:  (-5.53, -47.48)
Aeroporto:  (-21.8, -48.13)  WMO:  (-7.1, -34.86)
Aeroporto:  (-21.8, -48.13)  WMO:  (-23.31, -51.13)
Aeroporto:  (-21.8, -48.13)  WMO:  (-0.05, -51.11)
Aeroporto:  (-21.8, -48.13)  WMO:  (-0.05, -51.11)
Aeroporto:  (-21.8, -48.13)  WMO:  (-9.66, -35.7)
Aeroporto:  (-21.8, -48.13)  WMO:  (-3.1, -60.01)
Aeroporto:  (-21.8, -48.13)  W

Aeroporto:  (-23.03, -45.5)  WMO:  (-30.05, -51.16)
Aeroporto:  (-23.03, -45.5)  WMO:  (-8.05, -34.95)
Aeroporto:  (-23.03, -45.5)  WMO:  (-9.96, -67.8)
Aeroporto:  (-23.03, -45.5)  WMO:  (-22.89, -43.18)
Aeroporto:  (-23.03, -45.5)  WMO:  (-13.01, -38.53)
Aeroporto:  (-23.03, -45.5)  WMO:  (-2.53, -44.21)
Aeroporto:  (-23.03, -45.5)  WMO:  (-23.5, -46.61)
Aeroporto:  (-23.03, -45.5)  WMO:  (-3.83, -64.7)
Aeroporto:  (-23.03, -45.5)  WMO:  (-5.08, -42.81)
Aeroporto:  (-23.03, -45.5)  WMO:  (-19.73, -47.95)
Aeroporto:  (-23.03, -45.5)  WMO:  (-29.75, -57.08)
Aeroporto:  (-23.03, -45.5)  WMO:  (-20.31, -40.31)
O WMO 83781 é o mais próximo ao aeroporto de ICAO SBTA, com distância geoespacial de 124.92994436258834
Aeroporto:  (-9.35, -54.88)  WMO:  (-3.21, -52.21)
Aeroporto:  (-9.35, -54.88)  WMO:  (-10.95, -37.04)
Aeroporto:  (-9.35, -54.88)  WMO:  (-1.43, -48.43)
Aeroporto:  (-9.35, -54.88)  WMO:  (-19.93, -43.93)
Aeroporto:  (-9.35, -54.88)  WMO:  (2.82, -60.66)
Aeroporto:  (-9.35, -54.

ValueError: cannot convert float NaN to integer

In [9]:
#Remove dado sujo do rodapé do dataframe de aeroportos
dataframe_aeroportos = dataframe_aeroportos.loc[dataframe_aeroportos['Estação:']!='INPE']

In [10]:
#Recupera tuplas de aeroportos com o wmo tratado também por proximidade, não deixando wmos nulos.
tuplas_com_wmo_tratado = []
for index, aeroporto in dataframe_aeroportos.iterrows():
    icao
    try:
        icao = mapa_wmo_proximo_por_aeroporto[aeroporto['ICAO:']]
    except:
        icao = None
    if aeroporto['WMO:'] == '-' or icao is not None:
        aeroporto['WMO:'] = str(mapa_wmo_proximo_por_aeroporto[aeroporto['ICAO:']])
    else:
        aeroporto['WMO:'] = str(aeroporto['WMO:'])
    tuplas_com_wmo_tratado.append(aeroporto)
        
dataframe_aeroportos_com_wmo_tratado = DataFrame(tuplas_com_wmo_tratado)

In [11]:
#Realiza novamente o join, agora com todos os aeroportos possuindo wmo
df_resultante = merge(dataframe_aeroportos_com_wmo_tratado, dataframe_estacao_climatica, left_on = 'WMO:', right_on = 'Estacao', how = 'left')

In [12]:
df_resultante.loc[df_resultante['DirecaoVento'].isnull()]

,Altitude:,Estação:,ICAO:,Latitude:,Longitude:,País:,WMO:,,Data,DirecaoVento,...,Hora,Latitude,Longitude,PressaoAtmEstacao,TempBulboSeco,TempBulboUmido,UmidadeRelativa,VelocidadeVentoNebulosidade,DESCRIÇÃO,DESCSIMPLES


In [13]:
df_resultante['ICAO:'].unique()

array(['SBOI', 'SBBV', 'SBTS', 'SBAM', 'SBYA', 'SBMD', 'SBMQ', 'SBUA',
       'SWBC', 'SBMZ', 'SBBE', 'SBSN', 'SNCW', 'SBSL', 'SBPB', 'SBTF',
       'SBMN', 'SBEG', 'SBIC', 'SBHT', 'SBTU', 'SBFZ', 'SBFN', 'SBBC',
       'SBTT', 'SBUI', 'SBIH', 'SBMY', 'SBMA', 'SBIZ', 'SBCJ', 'SBTE',
       'SBMS', 'SBNT', 'SBER', 'SBEK', 'SBFO', 'SBCZ', 'SBLB', 'SBCI',
       'SBKG', 'SBJP', 'SBTK', 'SBBA', 'SBPV', 'SBAA', 'SBRF', 'SBWF',
       'SBRB', 'SBCC', 'SBAT', 'SBPL', 'SBUF', 'SBMO', 'SBXM', 'SBGM',
       'SBJR', 'SBPI', 'SBRQ', 'SBPN', 'SBPJ', 'SBAR', 'SBFS', 'SBSC',
       'SBSY', 'SBVH', 'SBSV', 'SBLP', 'SBXV', 'SBQV', 'SBIL', 'SBBW',
       'SBCY', 'SBGA', 'SBBR', 'SBAN', 'SBGO', 'SBMK', 'SBPS', 'SBCV',
       'SBUL', 'SBCR', 'SBCF', 'SBPR', 'SBUR', 'SBBH', 'SBLS', 'SBCG',
       'SBUP', 'SBBT', 'SBFC', 'SBPC', 'SBVT', 'SBRP', 'SBYS', 'SBAU',
       'SBLN', 'SBBQ', 'SBJF', 'SBCP', 'SBPP', 'SBPW', 'SBGW', 'SBDN',
       'SBKP', 'SBBU', 'SBSA', 'SBRS', 'SBGL', 'SBAF', 'SBME', 'SBRJ',
      

In [18]:
df_resultante['Data'][0]

'01/01/2015'

In [22]:
tuples = []

for index,row in df_resultante.iterrows():
    data_split = row['Data'].split('/')
    hora = row['Hora'][:2]
    row['Datetime'] = '{}-{}-{} {}:00:00'.format(data_split[2],data_split[1],data_split[0],hora)
    tuples.append(row)
    
df_resultante_tratado_data = DataFrame(tuples)

In [21]:
df_resultante_tratado_data

,Altitude:,Estação:,ICAO:,Latitude:,Longitude:,País:,WMO:,,Data,DirecaoVento,...,Hora,Latitude,Longitude,PressaoAtmEstacao,TempBulboSeco,TempBulboUmido,UmidadeRelativa,VelocidadeVentoNebulosidade,DESCRIÇÃO,DESCSIMPLES
0,39,OIAPOQUE AEROPORTO,SBOI,3.84,-51.82,BZ,82098,10,2015-01=01,0,...,0000,-0.05,-51.11,1010.4,26.4,23.6,77,0,NaN,NaN
1,39,OIAPOQUE AEROPORTO,SBOI,3.84,-51.82,BZ,82098,8,2015-01=01,0,...,1200,-0.05,-51.11,1012.1,25.2,29.2,92,0,NaN,NaN
2,39,OIAPOQUE AEROPORTO,SBOI,3.84,-51.82,BZ,82098,6,2015-01=01,7,...,1800,-0.05,-51.11,1008.7,30.4,26,69,2.2,NaN,NaN
3,39,OIAPOQUE AEROPORTO,SBOI,3.84,-51.82,BZ,82098,5,2015-01=02,5,...,0000,-0.05,-51.11,1010.2,28,24.8,74,3.3,NaN,NaN
4,39,OIAPOQUE AEROPORTO,SBOI,3.84,-51.82,BZ,82098,6,2015-01=02,5,...,1200,-0.05,-51.11,1011.4,27.4,25,80,1.2,NaN,NaN
5,39,OIAPOQUE AEROPORTO,SBOI,3.84,-51.82,BZ,82098,6,2015-01=02,7,...,1800,-0.05,-51.11,1009.2,32.2,25,54,3,NaN,NaN
6,39,OIAPOQUE AEROPORTO,SBOI,3.84,-51.82,BZ,82098,8,2015-01=03,5,...,0000,-0.05,-51.11,1010.3,28,24.6,73,3.3,NaN,NaN
7,39,OIAPOQUE AEROPORTO,SBOI,3.84,-51.82,BZ,82098,8,2015-01=03,5,...,1200,-0.05,-51.11,1010,27.2,24.8,82,3.3,NaN,NaN
8,39,OIAPOQUE AEROPORTO,SBOI,3.84,-51.82,BZ,82098,5,2015-01=03,9,...,1800,-0.05,-51.11,1008.1,32.4,25.8,58,2.3,NaN,NaN
9,39,OIAPOQUE AEROPORTO,SBOI,3.84,-51.82,BZ,82098,6,2015-01=04,5,...,0000,-0.05,-51.11,1010.1,29,25.2,72,1.6,NaN,NaN


In [ ]:
os.mkdir('result')
df_resultante_tratado_data.to_csv(path_or_buf='result/base_climatica.csv')